In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date
0,USA,211408,22878.0,4718.0,+665,8805.0,197885,5005.0,639.0,14.0,Jan 20,2020-04-01 18:18:15.283479
1,Italy,110574,4782.0,13155.0,+727,16847.0,80572,4035.0,1829.0,218.0,Jan 29,2020-04-01 18:18:15.283479
2,Spain,104118,8195.0,9387.0,+923,22647.0,72084,5872.0,2227.0,201.0,Jan 30,2020-04-01 18:18:15.283479
3,Germany,77921,6113.0,925.0,+150,18700.0,58296,3408.0,930.0,11.0,Jan 26,2020-04-01 18:18:15.283479
4,France,56989,4861.0,4032.0,+509,10934.0,42023,6017.0,873.0,62.0,Jan 23,2020-04-01 18:18:15.283479
...,...,...,...,...,...,...,...,...,...,...,...,...
200,Sierra Leone,2,1.0,0.0,0,0.0,2,0.0,0.3,0.0,Mar 30,2020-04-01 18:18:15.283479
201,Papua New Guinea,1,0.0,0.0,0,0.0,1,0.0,0.1,0.0,Mar 19,2020-04-01 18:18:15.283479
202,St. Vincent Grenadines,1,0.0,0.0,0,1.0,0,0.0,9.0,0.0,Mar 10,2020-04-01 18:18:15.283479
203,Timor-Leste,1,0.0,0.0,0,0.0,1,0.0,0.8,0.0,Mar 20,2020-04-01 18:18:15.283479


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Italy,110574,4782.0,13155.0,+727,16847.0,80572,4035.0,1829.0,218.0,Jan 29,2020-04-01 18:18:15.283479,12.070013,42.796626,30002.0,0.438471,0.118970,0.125119,0.014885,Last 8 columns are added by Roaya
175,China,81554,0.0,3312.0,0,76238.0,2004,466.0,57.0,2.0,Jan 10,2020-04-01 18:18:15.283479,103.819074,36.561765,79550.0,0.041634,0.040611,0.331751,0.013473,Last 8 columns are added by Roaya
2,Spain,104118,8195.0,9387.0,+923,22647.0,72084,5872.0,2227.0,201.0,Jan 30,2020-04-01 18:18:15.283479,-3.647550,40.244487,32034.0,0.293032,0.090157,0.133593,0.012044,Last 8 columns are added by Roaya
5,Iran,47593,2988.0,3036.0,+138,15473.0,29084,3871.0,567.0,36.0,Feb 18,2020-04-01 18:18:15.283479,54.274070,32.575033,18509.0,0.164028,0.063791,0.077189,0.004924,Last 8 columns are added by Roaya
4,France,56989,4861.0,4032.0,+509,10934.0,42023,6017.0,873.0,62.0,Jan 23,2020-04-01 18:18:15.283479,2.618787,47.824905,14966.0,0.269411,0.070750,0.062413,0.004416,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Benin,13,4.0,0.0,0,1.0,12,0.0,1.0,0.0,Mar 15,2020-04-01 18:18:15.283479,2.327853,9.641760,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
145,Saint Lucia,13,0.0,0.0,0,1.0,12,0.0,71.0,0.0,Mar 12,2020-04-01 18:18:15.283479,-60.969699,13.894795,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
146,Dominica,12,0.0,0.0,0,0.0,12,0.0,167.0,0.0,Mar 21,2020-04-01 18:18:15.283479,-61.357726,15.439470,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
148,Greenland,10,0.0,0.0,0,2.0,8,0.0,176.0,0.0,Mar 15,2020-04-01 18:18:15.283479,-41.341911,74.710513,2.0,0.000000,0.000000,0.000008,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":211408,"NewCases":22878.0,"TotalDeaths":4718.0,"NewDeaths":"+665","TotalRecovered":8805.0,"ActiveCases":197885,"CriticalCases":5005.0,"Tot\\u00a0Cases\\/1M pop":639.0,"Deaths\\/1M pop":14.0,"Reported1st case":"Jan 20","Date":1585765095283,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":13523.0,"ClosedDeathRate":0.3488870813,"CurrentDeathRate":0.0223170363,"Weight":0.0563956495,"DeathRateXWeight":0.0012585838,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":110574,"NewCases":4782.0,"TotalDeaths":13155.0,"NewDeaths":"+727","TotalRecovered":16847.0,"ActiveCases":80572,"CriticalCases":4035.0,"Tot\\u00a0Cases\\/1M pop":1829.0,"Deaths\\/1M pop":218.0,"Reported1st case":"Jan 29","Date":1585765095283,"Longitude":12.07001339,"Latitude":42.79662641,"ClosedCases":30002.0,"ClosedDeathRate":0.4384707686,"CurrentDeathRate":0.1189701015,"Weight":0.125118855,"DeathRateXWeight":0.0148854029,"notes":"Last 8 columns are added by Roaya"